# Building, Running and Pushing Docker Containers

> Before proceeding with the hands-on in this lesson, make sure you have completed the ones in the Dockerfile lesson first. We will use the Dockerfile created in that lesson throughout this notebook.

## Hands-On: Building Docker Images from Dockerfiles

To create a Docker image, you use the `docker build` command. This command reads the instructions in a Dockerfile and executes them step by step to construct the image. Each instruction in the Dockerfile results in a new image layer. Therefore, Docker images are constructed from multiple layers, and each layer represents a set of changes or instructions from the Dockerfile.

The basic syntax for building an image is as follows: `docker build -t <image_name>:<tag> <path_to_Dockerfile_directory>`.

- `-t`: Specifies the name and optional tag for the image. Tags provide a way to version your images (e.g., my-app:1.0).

- `<image_name>`: The name you want to give to your Docker image

- `<tag>`: An optional tag for versioning your image

- `<path_to_Dockerfile_directory>`: The directory where your Dockerfile is located

You can view all the options [here](https://docs.docker.com/engine/reference/commandline/build/).

For example, to build an image named `my-app` with the tag `v1.0` from a Dockerfile in the current directory, you would run: `docker build -t my-app:v1.0`.

> The typical naming convention for Docker images is `image_name:version`. Typically we specify the version as `latest` rather than manually writing out the semantic versioning label.

Going back to our previous scraper hands-on, to build the Docker image from the previously created Dockerfile, in the CLI, change the directory to `celebrity_example`. Then use the `build` command from Docker following the syntax:

In [ ]:
docker build -t celebrities:latest .

Since we are in the same directory as the Dockerfile, the Dockerfile path is simply a dot (`.`). To verify if the image was successfully created, you can run the following command:

In [ ]:
docker images # show our current images on this machine

<p align=center> <img src=images/Docker_images.png width=600> </p>

## Docker Image Building Techniques

### Docker Build Context

The *Docker build context* is the foundation upon which Docker images are constructed. It comprises a set of files and directories that Docker has access to during the image creation process. Understanding how the build context works is vital because it influences what gets included in the image and directly impacts image size and build speed.

> The build context is determined by the directory you specify when you run the `docker build` command. Everything within this directory and its subdirectories is part of the build context.

Consider a scenario where you have a directory with your application code, configuration files, and some large dataset. If you specify this directory as the build context, Docker will include all these files in the image, making it much larger than necessary.

### `.dockerignore` File

> The `.dockerignore` file is a key component of managing the Docker build context effectively. It is a plain text file in which you can define rules for excluding files and directories from the Docker build context, ensuring that only essential data is sent to the Docker daemon during image builds. This not only improves build performance but also reduces the risk of including unnecessary or sensitive information in the image.

The format of a `.dockerignore` file is straightforward. Each line typically represents a pattern or a rule, and Docker uses these rules to determine what should not be included when building the image. Here is an example of a basic `.dockerignore` file:

In this example, the `*.temp` and `*.log` patterns ensure that any temporary or log files are excluded. The `dev/` pattern excludes the entire `dev` directory, and the `secrets/` pattern ensures that any sensitive configuration files within the `secrets` directory are not included.

Understanding the scenarios in which you should use `.dockerignore` files is essential for efficient Docker image management. Here are some common use cases:

- **Excluding Development Artifacts**: When working on a development project, exclude files like build artifacts, IDE-specific files, and debug logs from the build context to keep the image clean and focused

- **Enhancing Security**: Exclude sensitive files such as credentials, private keys, and configuration files from the build context to prevent them from being inadvertently included in the Docker image

- **Minimizing Image Size**: Exclude files that are not required at runtime, like documentation, tests, or non-essential assets, to create leaner and more efficient Docker images

- **Improving CI/CD Pipeline Speed**: In CI/CD pipelines, minimizing the build context size can significantly reduce build times, ensuring faster deployments and integration testing

### Multi-Stage Builds

*Multi-stage builds* are a feature in Docker that enable you to create multiple stages (or intermediate images) within a single Dockerfile. Each stage represents a phase in the image-building process, and the final image is built by copying artifacts from one or more of these stages. This approach helps you achieve two important goals:

- **Efficiency**: Multi-stage builds allow you to optimize the size of the final image by including only what's necessary. Unnecessary build tools and dependencies can be discarded, resulting in leaner and more efficient images.

- **Modularity**: Dockerfiles can become complex as projects grow. Multi-stage builds improve modularity by breaking down the image creation process into distinct stages, making it easier to manage and maintain Dockerfiles.

Suppose you are developing a Python web application that relies on external dependencies managed by `pip`. To create an optimized production image, you can leverage multi-stage builds. Here's a breakdown of the Dockerfile:

``` dockerfile
# Stage 1: Build Dependencies
FROM python:3.9 as builder

WORKDIR /app

COPY requirements.txt .

# Install application dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Stage 2: Create Production Image
FROM python:3.9-slim

WORKDIR /app

# Copy only the application code and installed dependencies
COPY --from=builder /usr/local/lib/python3.9/site-packages /usr/local/lib/python3.9/site-packages
COPY . .

# Set environment variables, configure application settings, etc.

# Your application startup command
CMD ["python", "app.py"]
```

In this example:

- **Stage 1 (builder)**: This stage is responsible for installing application dependencies specified in `requirements.txt`. By separating this step, you ensure that only necessary dependencies are included in the final image.

- **Stage 2**: In this stage, you create the production image based on a lightweight Python image (`python:3.9-slim`). You copy the installed dependencies from the builder stage and then copy your application code into the image. This results in a production-ready image that contains only what is needed to run your application.

## Hands-On: Running Docker Containers

Running a Docker container is straightforward using the `docker run` command. Here's the basic syntax:

`docker run [OPTIONS] IMAGE[:TAG|@DIGEST] [COMMAND] [ARG...]`

To run the `celebrities image`, you would use the following command: `docker run celebrities`

This will throw an error because the script expects an input. However, at present, this is impossible because the image is running in a non-interactive mode. As a solution, we must add the options, `-i` and `-t`: `-i` will keep the STDIN open, and `-t` will make the process interactive.

<p align=center> <img src=images/Docker_run_error.png width=600> </p>

`docker run -it celebrities`

<p align=center> <img src=images/Docker_run.png width=600> </p>

There other common Docker container operations you may need to use. These include:

- **Stopping a Container**: To stop a running container gracefully, you can use the `docker stop` command followed by the container's ID or name `docker stop <container_id_or_name>`. You can obtain the image ID using the `docker images` command.

- **Removing a Container**: If you want to remove a stopped container, you can use the `docker rm` command with the container's ID or name: `docker rm <container_id_or_name>`

- **Listing Running Containers**: To see a list of running containers, you can run the `docker ps` command

- **Listing All Containers**: To see a list of all containers (including stopped ones), you can use the `-a` flag with docker `ps`

## Hands-On: Pushing Images to Docker Hub

Now that we have successfully build the `celebrities` image in the previous hands-on, the image can be used everywhere, regardless of the OS and dependencies installed. Additionally, you can distribute it globally using Docker Hub. To do this, you need to log in to your Docker Hub account using the CLI:

```
docker login
```

You will be prompted to enter your Docker Hub username and password. After entering your credentials, you should see a successful login message.

> Before pushing an image to Docker Hub, you need to tag it with the appropriate repository name and optionally specify a tag. The repository name typically follows the format `<username>/<image_name>`.

Use the `docker tag` command to add the repository name and tag to your image: `docker tag <image_id> <username>/<image_name>:<tag>`.

- `<image_id>`: The image ID of your existing Docker image. You can find this out by running `docker images`.
- `<username>/<image_name>`: The repository name on Docker Hub where you want to push the image
- `<tag>` (optional): A specific tag for versioning your image (e.g., v1.0). If not specified, it defaults to latest

Let's tag our previously created image:

```
docker tag 82a51cbd4876 ivanyingx/celebrities:v1
```
Above, `ivanyingx` is the username, which you should replace with yours, and `82a51cbd4876` is the image ID. Afterwards, confirm that the image has been properly built by running `docker images` once more.

<p align=center> <img src=images/Docker_tag.png width=800 height=80> </p>

Incidentally, it is also possible to confirm this information in Docker Desktop if you are on Mac or Windows:

<p align=center> <img src=images/Docker_Desktop.png width=1000 height=600> </p>

With the image tagged, you can now push it to Docker Hub using the `docker push` command: `docker push <username>/<image_name>:<tag>`. Which for this example will be: `docker push ivanyingx/celebrities:v1`.

To verify that your image has been uploaded, go to your Docker Hub account and check you can see the newly pushed image

<p align=center> <img src=images/Docker_Hub_example.png width=1000 height=350> </p>

If any other Docker Hub user wants to run your container, they can do so directly on their local machine. For example, in this case, you can run `ivanyingx`'s image as follows:

run `docker pull ivanyingx/celebrities` to download the image

and `docker run ivanyingx/celebrities` to run the image.

It is also possible to run `docker run ivanyingx/celebrities` directly, which will perform both operations.

Congratulations! You've successfully built and pushed your first Docker image.

## Key Takeaways

- Building Docker images involves creating Dockerfiles that define the environment, copy files, and run commands to create efficient and optimized containers
- Running Docker containers requires understanding the `docker run` command, managing container states, and effectively interacting with your containers
- Pushing Docker images to Docker Hub enables global accessibility, making your applications available to users and collaborators worldwide
- Docker's versatility empowers developers to create, deploy, and scale applications consistently, regardless of the underlying infrastructure